<a href="https://colab.research.google.com/github/Viroslav/Farukh_bot/blob/main/kaggle(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [2]:
train = pd.read_csv("/content/train.csv")
stores = pd.read_csv("/content/stores.csv")
oil = pd.read_csv("/content/oil.csv")
transactions = pd.read_csv("/content/transactions.csv")
holidays_events = pd.read_csv("/content/holidays_events.csv")

In [3]:
stores_sales = train.merge(stores).drop(['id'], axis = 1)
stores_sales

,date,store_nbr,family,sales,onpromotion,city,state,type,cluster
0,2013-01-01,1.0,AUTOMOTIVE,0.00000,0.0,Quito,Pichincha,D,13
1,2013-01-01,1.0,BABY CARE,0.00000,0.0,Quito,Pichincha,D,13
2,2013-01-01,1.0,BEAUTY,0.00000,0.0,Quito,Pichincha,D,13
3,2013-01-01,1.0,BEVERAGES,0.00000,0.0,Quito,Pichincha,D,13
4,2013-01-01,1.0,BOOKS,0.00000,0.0,Quito,Pichincha,D,13
...,...,...,...,...,...,...,...,...,...
214842,2013-04-30,9.0,POULTRY,367.71402,0.0,Quito,Pichincha,B,6
214843,2013-04-30,9.0,PREPARED FOODS,46.00000,0.0,Quito,Pichincha,B,6
214844,2013-04-30,9.0,PRODUCE,0.00000,0.0,Quito,Pichincha,B,6
214845,2013-04-30,9.0,SCHOOL AND OFFICE SUPPLIES,0.00000,0.0,Quito,Pichincha,B,6


In [4]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler
import math
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras import optimizers
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [5]:
X = stores_sales.drop(['sales'], axis = 1)

In [6]:
y = stores_sales.sales

In [7]:
X['date'] =pd.to_datetime(X['date'])
X.index= X['date']
X = X.drop(['date'], axis = 1)

In [8]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
X['family_cat']= labelencoder.fit_transform(X.family.values)
X['city_cat']= labelencoder.fit_transform(X.city.values)
X['state_cat']= labelencoder.fit_transform(X.state.values)
X['type_cat']= labelencoder.fit_transform(X.type.values)

In [9]:
X = X.drop(['family', 'city', 'state', 'type'], axis = 1)

In [10]:
class RegressionNet(torch.nn.Module):
    def __init__(self, n_hidden_neurons):
        super(RegressionNet,self).__init__()
        self.fc1 = torch.nn.Linear(in_features= X.shape[1], out_features=20, bias=True)      #1 слой, полносвязный
        self.act1 = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(in_features=20, out_features=20, bias=True)     #2 слой полносвязный
        self.act2 = torch.nn.ReLU()
        self.fc3 = torch.nn.Linear(in_features=20, out_features=1, bias=True)      #3 слой - выход

    def forward(self, x):
        x = self.fc1(x)
        x = self.act1(x)
        x = self.fc2(x)
        x = self.act2(x)
        x = self.fc3(x)
        return x

net = RegressionNet(100)

In [11]:
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=1.0e-3)

In [12]:
test_size = round(int(0.02 * X.shape[0]))
test_size

4296

In [13]:
X_test = X[-test_size:]

In [14]:
y_test = y[-test_size:]
y_test

210551      0.00000
210552    333.95300
210553    323.42902
210554      7.00000
210555      0.00000
            ...    
214842    367.71402
214843     46.00000
214844      0.00000
214845      0.00000
214846     14.73100
Name: sales, Length: 4296, dtype: float64

In [15]:
# creating tensor from targets_df 
X_train = torch.FloatTensor(X.values)
X_test = torch.FloatTensor(X_test.values)
y_train = torch.LongTensor(y.values)
y_test = torch.LongTensor(y_test.values)

In [ ]:
# ------Dataset preparation end--------:

# loss MSE
def loss(pred, target):
    squares = (pred - target) ** 2
    return squares.mean()

for epoch_index in range(1000):
    optimizer.zero_grad()

    y_pred = net.forward(X_train)
    loss_value = loss(y_pred, y_train)
    loss_value.backward()
    optimizer.step()

# Проверка осуществляется вызовом кода:
def metric(pred, target):
    return (pred - target).abs().mean()

print(metric(net.forward(X_test), y_test).item())